In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from src.cleantext import *

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential

# Import Yelp dataset

In [ ]:
data = pd.read_csv('../cap2/data/predicted_review.csv').drop(columns= ['business_id', 'user_id'])

In [ ]:
data['True(1)/Deceptive(0)'] = (data['True(1)/Deceptive(0)'] == 'True').astype(bool)

In [ ]:
type(data['True(1)/Deceptive(0)'][0])

**seperating to 2 dfs**

In [ ]:
true = data[data['True(1)/Deceptive(0)'] == 1]

In [ ]:
true.to_csv('true.csv')

In [ ]:
decep = data[data['True(1)/Deceptive(0)'] == 0]
decep.to_csv('decep.csv')

In [10]:
true = pd.read_csv('true.csv').drop(columns='Unnamed: 0')

In [11]:
decep = pd.read_csv('decep.csv').drop(columns='Unnamed: 0')

In [42]:
true.isnull().sum()

Review                  0
Stars                   0
True(1)/Deceptive(0)    0
dtype: int64

In [43]:
decep.isnull().sum()

Review                  0
Stars                   0
True(1)/Deceptive(0)    0
dtype: int64

Sample 100,000 data for faster processing

In [12]:
true_samp = true.sample(100000, random_state = 123)

In [13]:
decep_samp = decep.sample(100000, random_state = 123)

**pre processing the data**

clean text

In [14]:
true_samp.Review = cleanText(true_samp.Review)

In [15]:
decep_samp.Review = cleanText(decep_samp.Review)

tokenize

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/suchaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
true_samp.Review = true_samp['Review'].apply(word_tokenize)

In [18]:
decep_samp.Review = decep_samp['Review'].apply(word_tokenize)

Remove stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suchaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
stopwords_ = set(stopwords.words('english'))

In [21]:
true_samp.Review = true_samp.Review.apply(lambda x: [item for item in x if item not in stopwords_ ])

In [22]:
decep_samp.Review = decep_samp.Review.apply(lambda x: [item for item in x if item not in stopwords_ ])

Stemming

In [23]:
stemmer = SnowballStemmer("english")
true_samp.Review = true_samp.Review.apply(lambda x: [stemmer.stem(y) for y in x])
decep_samp.Review = decep_samp.Review.apply(lambda x: [stemmer.stem(y) for y in x])

Detokenized

In [24]:
true_samp.Review = true_samp.Review.apply(lambda x:' '.join([y for y in x ]))
decep_samp.Review = decep_samp.Review.apply(lambda x:' '.join([y for y in x ]))

In [25]:
true_samp.to_csv('true_samp.csv')
decep_samp.to_csv('decep_samp.csv')

# Data Loaded to CSV checkpoint

In [3]:
true_samp = pd.read_csv('true_samp.csv')
true_samp.drop(columns='Unnamed: 0', inplace = True)

In [4]:
decep_samp = pd.read_csv('decep_samp.csv')
decep_samp.drop(columns='Unnamed: 0', inplace = True)

Vectorize

In [5]:
def vectorize(X, max_features=None, ngram_range=None):
    '''
    X: Detokenized Column
    max_features: maximum features wanted (int)
    ngram_range: (n,n) 
    '''
    tfidf = TfidfVectorizer(max_features, ngram_range)
    doc_tfidf_matrix = tfidf.fit_transform(X).todense()
    vector = pd.DataFrame(doc_tfidf_matrix, columns = tfidf.get_feature_names())
    return vector

In [6]:
true_samp = true_samp[['Review', 'Stars']]

In [7]:
decep_samp = decep_samp[['Review','Stars']]

**use first 50k of sampled data for faster processing / able to keep track of the sample set**

In [8]:
true_samp.Review = true_samp.Review.fillna('None')

In [9]:
true_samp_vec = vectorize(true_samp[:50000]['Review'],max_features= 1000)

In [10]:
decep_samp.Review = decep_samp.Review.fillna('None')

In [11]:
decep_samp_vec = vectorize(decep_samp[:50000]['Review'], max_features= 1000)

Combine vector with other feature 

In [12]:
decep_samp_vec['Stars'] = decep_samp['Stars'][:50000]

In [13]:
decep_samp_vec

,34,aa,aaa,aaaaaaaamaz,aaaaaamaz,aaaaaand,aaaaamaz,aaaaauuuuttttttuuuuuuuummmmmnnnnnnnnnnnnnnn,aaaahmaz,aaaammmaaaz,...,zwar,zwei,zweibelring,zweit,zwiebelringen,zwischen,zydeco,zzeek,zzzzzzzzzz,Stars
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [14]:
true_samp_vec['Stars'] = true_samp['Stars'][:50000]

In [15]:
true_samp_vec.iloc[:, :-1].values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Save vectors to pickle files

In [17]:
true_samp_vec.to_pickle('data/true_samp_vec.pkl')

In [18]:
decep_samp_vec.to_pickle('data/decep_samp_vec.pkl')

# Vectorized frames data checkpoint

In [ ]:
true_samp_vec = pd.read_pickle('data/true_samp_vec.pkl)

In [ ]:
decep_samp_vec = pd.read_pickle('data/decep_samp_vec.pkl')

split

In [ ]:
X_true_train, X_true_test = train_test_split(true_samp_vec.iloc[:, :-1].values, test_size = 0.2, random_state = 123)

# First Simple Auto Enconding Model

In [ ]:
# #model without rating feature
# model = Sequential()
# model.add(Dense(25, input_dim=X_true_train.shape[1], activation='relu'))
# model.add(Dense(3, activation='relu'))
# model.add(Dense(25, activation='relu'))
# model.add(Dense(X_true_train.shape[1])) # Multiple output neurons
# model.compile(loss='mean_squared_error', optimizer='adam')
# model.fit(X_true_train,X_true_train,verbose=1,epochs=100)

model without rating feature, epoch = 5

In [16]:
model = Sequential()
model.add(Dense(25, input_dim=X_true_train.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(X_true_train.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_true_train,X_true_train,verbose=1,epochs=5)

Train on 40000 samples
Epoch 1/5
40000/40000 [==============================] - 47s 1ms/sample - loss: 1.8347e-05
Epoch 2/5
40000/40000 [==============================] - 48s 1ms/sample - loss: 1.8350e-05
Epoch 3/5
40000/40000 [==============================] - 49s 1ms/sample - loss: 1.8352e-05
Epoch 4/5
40000/40000 [==============================] - 50s 1ms/sample - loss: 1.8353e-05
Epoch 5/5
40000/40000 [==============================] - 46s 1ms/sample - loss: 1.8353e-05


In [18]:
pred = model.predict(X_true_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,X_true_test))

In [19]:
print(f"Insample True Reviews Score (RMSE): {score1}".format(score1))

Insample True Reviews Score (RMSE): 0.004284969917401188


In [ ]:
pred = model.predict(true_samp_vec.iloc[:, :-1].values)
score2 = np.sqrt(metrics.mean_squared_error(pred,true_samp_vec.iloc[:, :-1].values))
print(f"Out of Sample True Reviews Score (RMSE): {score2}")

In [ ]:
pred = model.predict(decep_samp_vec.iloc[:, :-1].values)
score3 = np.sqrt(metrics.mean_squared_error(pred,decep_samp_vec.iloc[:, :-1].values))
print(f"Deceptive Reviews Score (RMSE): {score3}")

model with rating feature, epoch = 5

In [ ]:
#model with rating feature
model = Sequential()
model.add(Dense(25, input_dim=X_true_train.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(X_true_train.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam', metrics= )
model.fit(X_true_train,X_true_train,verbose=1,epochs=5)

In [ ]:
pred = model1.predict(X_true_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,X_true_test))
pred = model1.predict(true_samp_vec.values)
score2 = np.sqrt(metrics.mean_squared_error(pred,true_samp_vec.values))
pred = model1.predict(decep_samp_vec.values)
score3 = np.sqrt(metrics.mean_squared_error(pred,decep_samp_vec.values))
print(f"Insample True Reviews Score (RMSE): {score1}".format(score1))
print(f"Out of Sample True Reviews Score (RMSE): {score2}")
print(f"Deceptive Reviews Score (RMSE): {score3}")